In [1]:
# Needs to be run when session times out
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes datasets huggingface_hub

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-phpb1gin/unsloth_207bca4a6bb24908a84fc5d414935b59
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-phpb1gin/unsloth_207bca4a6bb24908a84fc5d414935b59
  Resolved https://github.com/unslothai/unsloth.git to commit 8dc0561ec0776fcc49d8a406c8a0acf295bd561a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import csv
from datasets import Dataset, load_dataset
from unsloth import FastLanguageModel
import torch
import random
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, TextStreamer, AutoTokenizer

Google Sheets integration

LLAMA2 INFERENCE - BASE MODEL FOR TEXT GENERATION
Llama2-base is used as a baseline to compare the fine-tuned model's perforomance

In [3]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-2-7b-bnb-4bit")

model = AutoModelForCausalLM.from_pretrained("unsloth/llama-2-7b-bnb-4bit")

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

INFERENCE CODE FOR MODEL FINE-TUNED FOR 1 EPOCH

In [ ]:
config = PeftConfig.from_pretrained("MMCHO/llama2_lpr_4")
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-2-7b-bnb-4bit")

# The base model is loaded from unsloth and the adapter config is added to it because we are not updating all model parameters
# The saved model does not config.json (base model params). So that is appended for inference
base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-2-7b-bnb-4bit")
model = PeftModel.from_pretrained(base_model, "MMCHO/llama2_lpr_4")

INFERENCE CODE FOR MODEL FINE-TUNED FOR 3 EPOCHS

In [ ]:
config = PeftConfig.from_pretrained("tuhinatripathi/llama2_lpr_5")
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-2-7b-bnb-4bit")

base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-2-7b-bnb-4bit")
model = PeftModel.from_pretrained(base_model, "tuhinatripathi/llama2_lpr_5")

To extract the rewrite prompt, we analyse the pattern of generated data and realize the model always generates the rewrite prompt between the last and second last newline characters. This split was not acheivable through sentence tokenization so we make a hack to extract the rewrite prompt.

In [4]:
def extract_prompt(text):
  # last '\n'
  last_newline_index = text.rfind("\n")

  # Second to last '\n'
  second_to_last_newline_index = text.rfind("\n", 0, last_newline_index)

  # Extract the rewrite prompt
  desired_text = text[second_to_last_newline_index + 1:last_newline_index]

  return desired_text

The data is structured similar to the train script.

In [5]:
llama_prompt = """Analyze the differences between the "Original Text" and "Rewritten Text", and infer the specific "Rewrite Prompt" that was likely used to rewrite/transform/improve the text in this way. Output only the "Rewrite Prompt" in the following format \n Rewrite Prompt: <predicted rewrite prompt> \n\n
### Original Text:
{}

### Rewriten Text:
{}
###
"""
def formatting_prompts_func(test_data):
    # Define a function to format each example in the dataset

    inputs = test_data["OriginalText"]
    outputs = test_data["TargetText"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = llama_prompt.format(input, output)
        texts.append(text)
    return { "text" : texts, }
pass

In [6]:
# Test data for inference loaded from huggingFace
data = load_dataset("tuhinatripathi/llm-prompt-recovery", split = 'test')
test_data = data.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [8]:
# Enables two times faster inference
FastLanguageModel.for_inference(model)

The respective model tokenizer is used to prepare the input text for inference.
A separate function is used to extract the rewrite prompt from generated code.

In [9]:
preds = []
for data in test_data:
  inputs = tokenizer(data["text"], return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
  pred = extract_prompt(tokenizer.batch_decode(outputs)[0])
  preds.append(pred)
  print(pred)

###
###
### Original Text:
### 
### Original Text:
### Rewriten Text:
### Original Text:

###
### Rewritten Text:


We save all the predictions in a csv using the function below

In [ ]:
import csv

def create_file(output_file, target, predicted):
	print("Create CSV function called")

	# Specify the CSV file path
	csv_file_path = output_file

	# Write extracted fields to CSV file
	with open(csv_file_path, 'w', newline='') as csvfile:
		fieldnames = ['TargetPrompt', 'PredictedPrompt']
		writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

		# Write header
		writer.writeheader()

		# Write data
		for i, entry in enumerate(target):
			# entry_row = json.loads(entry)
			writer.writerow({
				'TargetPrompt': entry,
				'PredictedPrompt': predicted[i],
			})

	print(f'CSV file "{csv_file_path}" has been created.')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = '/content/drive/MyDrive/results.csv'
create_file(filename, test_data[:]["RewritePrompt"], preds)